# 챗봇 전처리 클래스

In [8]:
# 챗봇 전처리 클래스 
from konlpy.tag import Komoran
import pickle

class Preprocess:
    def __init__(self, word2index_dic ='', userdic=None):   
        # 단어사전 불러오기     
        if(word2index_dic !=''):
            f = open(word2index_dic, "rb")
            self.word_index = pickle.load(f)
            f.close()
        else:
            self.word_index=None
            
        self.komoran = Komoran(userdic=userdic)
        self.exclusion_tags = [
            'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 
            'JX', 'JC',
            'SF', 'SP', 'SS', 'SE', 'SO',
            'EP', 'EF', 'EC', 'ETN', 'ETM',
            'XSN', 'XSV', 'XSA','VV'
        ]
        # 관계언(격조사,보조사,접속조사) 
        # 기호(마침표,물음표,느낌표,쉼포,가운뎃점,콜론,빗금,따옴표,괄호표,줄표,줄임표,붙임표)
        # 의존형태(어미,접두사,접미사)
        # 더 추가가능!!! 동사 제거해볼까 ????? 
        
    # 형태소 분석기 POS 태거
    def pos(self, sentence):
        return self.komoran.pos(sentence)

    # 불용어 제거 후, 필요한 품사 정보만 가져오기
    def get_keywords(self, pos, without_tag=False):
        f = lambda x: x in self.exclusion_tags
        word_list = []
        for p in pos:
            if f(p[1]) is False:
                word_list.append(p if without_tag is False else p[0])
        return word_list
    
    # 키워드를 단어 인덱스 스퀀스로 변환
    def get_wordidx_sequence(self, keywords):
        if self.word_index is None:
            return []
        w2i = []
        for word in keywords:
            try:
                w2i.append(self.word_index[word])
            except KeyError:
                #해당 단어가 사전에 없는 경우 OOV 처리
                w2i.append(self.word_index['OOV'])
        return w2i



# 챗봇 전처리 클래스 테스트

In [9]:
# sent = '3년제 전문학사 졸업하고 학점은행제를 통해서 학사학위를 받을 예정인데 학력기준을 충족하나요?'
# p = Preprocess(userdic='user_dic.tsv')

# pos = p.pos(sent)

# ret = p.get_keywords(pos, without_tag=False)
# print(ret)

# ret = p.get_keywords(pos, without_tag=True)
# print(ret)


In [10]:
# # 말뭉치 데이터 읽어오기
# def read_corpus_data(filename):
#     with open(filename, 'r', encoding="utf-8") as f:
#         data = [line.split('\t') for line in f.read().splitlines()]
#     return data

# corpus_data = read_corpus_data('raw_corpus.txt')

# p = Preprocess()
# dict = []

# for c in corpus_data:
#     pos = p.pos(c[0])
#     keywords = p.get_keywords(pos, without_tag=True)
#     for k in keywords:
#         dict.append(k) # 키워드만 추출한 사전
        
# tokenizer = preprocessing.text.Tokenizer(oov_token='OOV')
# tokenizer.fit_on_texts(dict)
# word_index = tokenizer.word_index
        
# print(word_index)


# 단어 사전 생성 
#### raw_chatbot_dict.bin   id, document, lable column


In [11]:
# from tensorflow.keras import preprocessing
# import pickle

# # 말뭉치 데이터 읽어오기
# def read_corpus_data(filename):
#     with open(filename, 'r', encoding="utf-8") as f:
#         data = [line.split('\t') for line in f.read().splitlines()]
#     return data


# # 말뭉치 데이터 가져오기
# corpus_data = read_corpus_data('raw_corpus.txt')


# # 망뭉치 데이터에서 키워드만 추출해서 사전 리스트 생성
# p = Preprocess()
# dict = []
# for c in corpus_data:
#     pos = p.pos(c[0])
#     keywords = p.get_keywords(pos, without_tag=True)
#     for k in keywords:
#         dict.append(k) # 키워드만 추출한 사전

# # 사전에 사용될 word2index 생성
# # 사전의 첫번 째 인덱스에는 OOV 사용
# tokenizer = preprocessing.text.Tokenizer(oov_token='OOV')
# tokenizer.fit_on_texts(dict)
# word_index = tokenizer.word_index

# # 사전 파일 생성
# f = open("raw_chatbot_dict.bin", "wb")
# try:
#     pickle.dump(word_index, f)
# except Exception as e:
#     print(e)
# finally:
#     f.close()


# 단어 사전 테스트

In [12]:
# import pickle

# # 단어 사전 불러오기
# f = open("raw_chatbot_dict.bin", "rb")
# word_index = pickle.load(f)
# f.close()

# sent = "MOOC 기초과정 학습을 따로 진행할 수 있는 방법이 있을지 궁금합니다."

# # 전처리 객체 생성
# p = Preprocess(userdic='user_dic.tsv')

# # 형태소분석기 실행
# pos = p.pos(sent)

# # 품사 태그 없이 키워드 출력
# keywords = p.get_keywords(pos, without_tag=True)
# for word in keywords:
#     try:
#         print(word, word_index[word])
#     except KeyError:
#         # 해당 단어가 사전에 없는 경우, OOV 처리
#         print(word, word_index['OOV'])



# 글로벌 파라미터 정보( 챗봇 엔진 소스 전역에서 사용할 파라미터 정보)

In [13]:
# 단어 시퀀스 벡터 크기
MAX_SEQ_LEN = 15

def GlobalParams():
    global MAX_SEQ_LEN

In [14]:
# # 필요한 모듈 임포트
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras import preprocessing
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate


# # 데이터 읽어오기
# train_file = "raw_total_train_data.csv" # 우리데이터로 수정 !!! 
# data = pd.read_csv(train_file, delimiter=',')
# queries = data['query'].tolist()
# intents = data['intent'].tolist()

# p = Preprocess(word2index_dic='raw_chatbot_dict.bin', userdic='user_dic.tsv') # 우리 데이터로 수정 !!!

# # 단어 시퀀스 생성
# sequences = []
# for sentence in queries:
#     pos = p.pos(sentence)
#     keywords = p.get_keywords(pos, without_tag=True)
#     seq = p.get_wordidx_sequence(keywords)
#     sequences.append(seq)
    
# padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

# # (105658, 15)
# print(padded_seqs.shape)
# print(len(intents)) #105658

# 의도 분류 모델 학습
#### chat_intent_model.h5

In [15]:
# # 필요한 모듈 임포트
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras import preprocessing
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate


# # 데이터 읽어오기
# train_file = "raw_total_train_data.csv" # 우리데이터로 수정 !!! 
# # train_file = "models/intent/total_train_data.csv"
# data = pd.read_csv(train_file, delimiter=',')
# queries = data['query'].tolist()
# intents = data['intent'].tolist()

# p = Preprocess(word2index_dic='raw_chatbot_dict.bin', userdic='user_dic.tsv') # 우리 데이터로 수정 !!!

# # 단어 시퀀스 생성
# sequences = []
# for sentence in queries:
#     pos = p.pos(sentence)
#     keywords = p.get_keywords(pos, without_tag=True)
#     seq = p.get_wordidx_sequence(keywords)
#     sequences.append(seq)


# # 단어 인덱스 시퀀스 벡터 ○2
# # 단어 시퀀스 벡터 크기
# # from config.GlobalParams import MAX_SEQ_LEN
# padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

# # (105658, 15)
# print(padded_seqs.shape)
# print(len(intents)) #105658

# # 학습용, 검증용, 테스트용 데이터셋 생성 ○3
# # 학습셋:검증셋:테스트셋 = 7:2:1
# ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents))
# ds = ds.shuffle(len(queries))

# train_size = int(len(padded_seqs) * 0.7)
# val_size = int(len(padded_seqs) * 0.2)
# test_size = int(len(padded_seqs) * 0.1)

# train_ds = ds.take(train_size).batch(20)
# val_ds = ds.skip(train_size).take(val_size).batch(20)
# test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

# # 하이퍼 파라미터 설정
# dropout_prob = 0
# EMB_SIZE = 128
# EPOCH = 10
# VOCAB_SIZE = len(p.word_index) + 1 #전체 단어 개수


# # CNN 모델 정의  ○4
# input_layer = Input(shape=(MAX_SEQ_LEN,))
# embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
# dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

# conv1 = Conv1D(
#     filters=128,
#     kernel_size=3,
#     padding='valid',
#     activation=tf.nn.relu)(dropout_emb)
# pool1 = GlobalMaxPool1D()(conv1)

# conv2 = Conv1D(
#     filters=128,
#     kernel_size=4,
#     padding='valid',
#     activation=tf.nn.relu)(dropout_emb)
# pool2 = GlobalMaxPool1D()(conv2)

# conv3 = Conv1D(
#     filters=128,
#     kernel_size=5,
#     padding='valid',
#     activation=tf.nn.relu)(dropout_emb)
# pool3 = GlobalMaxPool1D()(conv3)

# # 3,4,5gram 이후 합치기
# concat = concatenate([pool1, pool2, pool3])

# hidden = Dense(128, activation=tf.nn.relu)(concat)
# dropout_hidden = Dropout(rate=dropout_prob)(hidden)
# logits = Dense(5, name='logits')(dropout_hidden)
# predictions = Dense(5, activation=tf.nn.softmax)(logits)


# # 모델 생성  ○5
# model = Model(inputs=input_layer, outputs=predictions)
# model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


# # 모델 학습 ○6
# model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)


# # 모델 평가(테스트 데이터 셋 이용) ○7
# loss, accuracy = model.evaluate(test_ds, verbose=1)
# print('Accuracy: %f' % (accuracy * 100))
# print('loss: %f' % (loss))


# # 모델 저장  ○8
# model.save('real_chat.h5') # 우리 데이터 모델로 이름 변경 !!! 


# 의도 분류 모듈 생성

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing


# 의도 분류 모델 모듈
class IntentModel:
    def __init__(self, model_name, proprocess):

        # 의도 클래스 별 레이블
        self.labels = {0 : "인사", 1 : "사전학습안내", 2 : "서류제출안내", 3 : "운영일정안내", 4 : "지원자격안내" }
        # 의도 분류 모델 불러오기
        self.model = load_model(model_name)

        # 챗봇 Preprocess 객체
        self.p = proprocess


    # 의도 클래스 예측
    def predict_class(self, query):
        # 형태소 분석
        pos = self.p.pos(query)

        # 문장내 키워드 추출(불용어 제거)
        keywords = self.p.get_keywords(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]

        # 단어 시퀀스 벡터 크기
        # from config.GlobalParams import MAX_SEQ_LEN

        # 패딩처리
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

        predict = self.model.predict(padded_seqs)
        predict_class = tf.math.argmax(predict, axis=1)
        return predict_class.numpy()[0]


# IntentModel 클래스 태스트 

In [17]:
# # from models.intent.IntentModel import IntentModel

# p = Preprocess(word2index_dic='chatbot_dict.bin', userdic='user_dic.tsv')

# intent = IntentModel(model_name='real_chat.h5', proprocess=p)
# query = "필기시험 비대면으로 치나요?"
# predict = intent.predict_class(query)
# predict_label = intent.labels[predict]

# print("질문 :",query)
# print("의도 예측 클래스 : ", predict)
# print("의도 예측 레이블 : ", predict_label)



# 개체명 인식 모델 학습
#### ner_model.h5 -> change 

In [18]:
# import matplotlib.pyplot as plt
# import tensorflow as tf
# from tensorflow.keras import preprocessing
# from sklearn.model_selection import train_test_split
# import numpy as np

# # 학습 파일 불러오기
# def read_file(file_name):
#     sents = []
#     with open(file_name, 'r', encoding='utf-8') as f:
#         lines = f.readlines()
#         for idx, l in enumerate(lines):
#             if l[0] == ';' and lines[idx + 1][0] == '$':
#                 this_sent = []
#             elif l[0] == '$' and lines[idx - 1][0] == ';':
#                 continue
#             elif l[0] == '\n':
#                 sents.append(this_sent)
#             else:
#                 this_sent.append(tuple(l.split()))
#     return sents

# p = Preprocess(word2index_dic='raw_chatbot_dict.bin', userdic='user_dic.tsv') # 우리 데이터로 수정 !!!

# # 학습용 말뭉치 데이터를 불러옴
# corpus = read_file('models/ner/ner_train.txt') # 우리 데이터로 수정 !!!

# # 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
# sentences, tags = [], []
# for t in corpus:
#     tagged_sentence = []
#     sentence, bio_tag = [], []
#     for w in t:
#         tagged_sentence.append((w[1], w[3]))
#         sentence.append(w[1])
#         bio_tag.append(w[3])
    
#     sentences.append(sentence)
#     tags.append(bio_tag)


# print("샘플 크기 : \n", len(sentences))
# print("0번 째 샘플 단어 시퀀스 : \n", sentences[0])
# print("0번 째 샘플 bio 태그 : \n", tags[0])
# print("샘플 단어 시퀀스 최대 길이 :", max(len(l) for l in sentences))
# print("샘플 단어 시퀀스 평균 길이 :", (sum(map(len, sentences))/len(sentences)))

# # 토크나이저 정의
# tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보는 lower=False 소문자로 변환하지 않는다.
# tag_tokenizer.fit_on_texts(tags)

# # 단어사전 및 태그 사전 크기
# vocab_size = len(p.word_index) + 1
# tag_size = len(tag_tokenizer.word_index) + 1
# print("BIO 태그 사전 크기 :", tag_size)
# print("단어 사전 크기 :", vocab_size)

# # 학습용 단어 시퀀스 생성
# x_train = [p.get_wordidx_sequence(sent) for sent in sentences]
# y_train = tag_tokenizer.texts_to_sequences(tags)

# index_to_ner = tag_tokenizer.index_word # 시퀀스 인덱스를 NER로 변환 하기 위해 사용
# index_to_ner[0] = 'PAD'

# # 시퀀스 패딩 처리
# max_len = 40
# x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
# y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)

# # 학습 데이터와 테스트 데이터를 8:2의 비율로 분리
# x_train, x_test, y_train, y_test = train_test_split(x_train, y_train,
#                                                     test_size=.2,
#                                                     random_state=1234)

# # 출력 데이터를 one-hot encoding
# y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
# y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)

# print("학습 샘플 시퀀스 형상 : ", x_train.shape)
# print("학습 샘플 레이블 형상 : ", y_train.shape)
# print("테스트 샘플 시퀀스 형상 : ", x_test.shape)
# print("테스트 샘플 레이블 형상 : ", y_test.shape)


# # 모델 정의 (Bi-LSTM)
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
# from tensorflow.keras.optimizers import Adam

# model = Sequential()
# model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
# model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
# model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
# model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
# model.fit(x_train, y_train, batch_size=128, epochs=10)

# print("평가 결과 : ", model.evaluate(x_test, y_test)[1])
# model.save('ner_model.h5') # chat_ner로 수정 !!!


# # 시퀀스를 NER 태그로 변환
# def sequences_to_tag(sequences):  # 예측값을 index_to_ner를 사용하여 태깅 정보로 변경하는 함수.
#     result = []
#     for sequence in sequences:  # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
#         temp = []
#         for pred in sequence:  # 시퀀스로부터 예측값을 하나씩 꺼낸다.
#             pred_index = np.argmax(pred)  # 예를 들어 [0, 0, 1, 0 ,0]라면 1의 인덱스인 2를 리턴한다.
#             temp.append(index_to_ner[pred_index].replace("PAD", "O"))  # 'PAD'는 'O'로 변경
#         result.append(temp)
#     return result



# F1-score CHECK

In [19]:
# # f1 스코어 계산을 위해 사용
# from seqeval.metrics import f1_score, classification_report


# # 테스트 데이터셋의 NER 예측
# y_predicted = model.predict(x_test)
# pred_tags = sequences_to_tag(y_predicted) # 예측된 NER
# test_tags = sequences_to_tag(y_test)    # 실제 NER

# # F1 평가 결과
# print(classification_report(test_tags, pred_tags))
# print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))

# 개체명 인식 모듈 생성 --> 우리껄로 바꿔야댐 

In [20]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing


# 개체명 인식 모델 모듈
class NerModel:
    def __init__(self, model_name, proprocess):

        # BIO 태그 클래스 별 레이블
        self.index_to_ner = {1: 'O', 2: 'B_DT', 3: 'B_FOOD', 4: 'I', 5: 'B_OG', 6: 'B_PS', 7: 'B_LC', 8: 'NNP', 9: 'B_TI', 0: 'PAD'}

        # 의도 분류 모델 불러오기
        self.model = load_model(model_name)

        # 챗봇 Preprocess 객체
        self.p = proprocess


    # 개체명 클래스 예측
    def predict(self, query):
        # 형태소 분석
        pos = self.p.pos(query)

        # 문장내 키워드 추출(불용어 제거)
        keywords = self.p.get_keywords(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]

        # 패딩처리
        max_len = 40
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, padding="post", value=0, maxlen=max_len)

        predict = self.model.predict(np.array([padded_seqs[0]]))
        predict_class = tf.math.argmax(predict, axis=-1)

        tags = [self.index_to_ner[i] for i in predict_class.numpy()[0]]
        return list(zip(keywords, tags))

    def predict_tags(self, query):
        # 형태소 분석
        pos = self.p.pos(query)

        # 문장내 키워드 추출(불용어 제거)
        keywords = self.p.get_keywords(pos, without_tag=True)
        sequences = [self.p.get_wordidx_sequence(keywords)]

        # 패딩처리
        max_len = 40
        padded_seqs = preprocessing.sequence.pad_sequences(sequences, padding="post", value=0, maxlen=max_len)

        predict = self.model.predict(np.array([padded_seqs[0]]))
        predict_class = tf.math.argmax(predict, axis=-1)

        tags = []
        for tag_idx in predict_class.numpy()[0]:
            if tag_idx == 1: continue
            tags.append(self.index_to_ner[tag_idx])

        if len(tags) == 0: return None
        return tags

# NerModel 객체 사용 테스트

In [21]:
# from models.ner.NerModel import NerModel

# p = Preprocess(word2index_dic='chatbot_dict.bin', userdic='utils/user_dic.tsv')


# ner = NerModel(model_name='ner_model.h5', proprocess=p) # chat_ner로 수정 !!!
# query = '안녕하세요 제 이름은 포림이에요'
# predicts = ner.predict(query)
# tags = ner.predict_tags(query)
# print(predicts)
# print(tags)



# 여기서부터 서버 연동하는거 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# 데이터베이스 제어 모듈 생성

In [22]:
import pymysql
import pymysql.cursors
import logging


class Database:
    '''
    database 제어
    '''

    def __init__(self, host, user, password, db_name, charset='utf8'):
        self.host = host
        self.user = user
        self.password = password
        self.charset = charset
        self.db_name = db_name
        self.conn = None

    # DB 연결
    def connect(self):
        if self.conn != None:
            return

        self.conn = pymysql.connect(
            host=self.host,
            user=self.user,
            password=self.password,
            db=self.db_name,
            charset=self.charset
        )

    # DB 연결 닫기
    def close(self):
        if self.conn is None:
            return

        if not self.conn.open:
            self.conn = None
            return
        self.conn.close()
        self.conn = None

    # SQL 구문 실행
    def execute(self, sql):
        last_row_id = -1
        try:
            with self.conn.cursor() as cursor:
                cursor.execute(sql)
            self.conn.commit()
            last_row_id = cursor.lastrowid
            # logging.debug("excute last_row_id : %d", last_row_id)
        except Exception as ex:
            logging.error(ex)

        finally:
            return last_row_id

    # SELECT 구문 실행 후, 단 1개의 데이터 ROW만 불러옴
    def select_one(self, sql):
        result = None

        try:
            with self.conn.cursor(pymysql.cursors.DictCursor) as cursor:
                cursor.execute(sql)
                result = cursor.fetchone()
        except Exception as ex:
            logging.error(ex)

        finally:
            return result

    # SELECT 구문 실행 후, 전체 데이터 ROW만 불러옴
    def select_all(self, sql):
        result = None

        try:
            with self.conn.cursor(pymysql.cursors.DictCursor) as cursor:
                cursor.execute(sql)
                result = cursor.fetchall()
        except Exception as ex:
            logging.error(ex)

        finally:
            return result

# 챗봇 답변 검색 모듈

In [23]:
class FindAnswer:
    def __init__(self, db):
        self.db = db

    # 검색 쿼리 생성
    def _make_query(self, intent_name, ner_tags):
        sql = "select * from chatbot" ## 바꿔야 할듯? 
        if intent_name != None and ner_tags == None:
            sql = sql + " where intent='{}' ".format(intent_name)

        elif intent_name != None and ner_tags != None:
            where = ' where intent="%s" ' % intent_name
            if (len(ner_tags) > 0):
                where += 'and ('
                for ne in ner_tags:
                    where += " ner like '%{}%' or ".format(ne)
                where = where[:-3] + ')'
            sql = sql + where

        # 동일한 답변이 2개 이상인 경우, 랜덤으로 선택
        sql = sql + " order by rand() limit 1"
        return sql

    # 답변 검색
    def search(self, intent_name, ner_tags):
        # 의도명, 개체명으로 답변 검색
        sql = self._make_query(intent_name, ner_tags)
        answer = self.db.select_one(sql)

        # 검색되는 답변이 없으면 의도명만 검색
        if answer is None:
            sql = self._make_query(intent_name, None)
            answer = self.db.select_one(sql)

        return (answer['answer'], answer['answer_image'])

    # NER 태그를 실제 입력된 단어로 변환
    def tag_to_word(self, ner_predicts, answer):
        for word, tag in ner_predicts:

            # 변환해야하는 태그가 있는 경우 추가
            if tag == 'B_FOOD' or tag == 'B_DT' or tag == 'B_TI':
                answer = answer.replace(tag, word)

        answer = answer.replace('{', '')
        answer = answer.replace('}', '')
        return answer

# 챗봇 엔진 동작 테스트

In [24]:
from config.DatabaseConfig import *
import warnings
warnings.filterwarnings("ignore")

# 전처리 객체 생성
p = Preprocess(word2index_dic='raw_chatbot_dict.bin', userdic='user_dic.tsv')

# 질문/답변 학습 디비 연결 객체 생성
db = Database(
    host='localhost', user='root', password='aiproject', db_name='chatbot'
)
db.connect()    # 디비 연결

# 원문
query = "사전학습을 필수적으로 들어야하나요?"
# query = "화자의 질문 의도를 파악합니다."
# query = "안녕하세요"
# query = "자장면 주문할게요"

# 의도 파악
# from models.intent.IntentModel import IntentModel
intent = IntentModel(model_name='chat_intent_model.h5', proprocess=p)
predict = intent.predict_class(query)
intent_name = intent.labels[predict]

# 개체명 인식
# from models.ner.NerModel import NerModel
ner = NerModel(model_name='ner_model.h5', proprocess=p)
predicts = ner.predict(query)
ner_tags = ner.predict_tags(query)

print("질문 : ", query)
print("=" * 40)
print("의도 파악 : ", intent_name)
print("개체명 인식 : ", predicts)
print("답변 검색에 필요한 NER 태그 : ", ner_tags)
print("=" * 40)

# 답변 검색
# from utils.FindAnswer import FindAnswer

try:
    f = FindAnswer(db)
    answer_text, answer_image = f.search(intent_name, ner_tags)
    answer = f.tag_to_word(predicts, answer_text)
except:
    answer = "죄송해요 무슨 말인지 모르겠어요"

print("답변 : ", answer)

db.close() # 디비 연결 끊음


In [ ]:
error !!!!!!!

# 챗봇 서버 모듈 

In [25]:
import socket

class BotServer:
    def __init__(self, srv_port, listen_num):
        self.port = srv_port
        self.listen = listen_num
        self.mySock = None

    # sock 생성
    def create_sock(self):
        self.mySock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.mySock.bind(("0.0.0.0", int(self.port)))
        self.mySock.listen(int(self.listen))
        return self.mySock

    # client 대기
    def ready_for_client(self):
        return self.mySock.accept()

    # sock 반환
    def get_sock(self):
        return self.mySock


# 챗봇 엔진 서버 메인 프로그램 

In [26]:
import threading
import json

from models.intent.IntentModel import IntentModel
from models.ner.NerModel import NerModel


# 전처리 객체 생성
p = Preprocess(word2index_dic='raw_chatbot_dict.bin', userdic='user_dic.tsv')

# 의도 파악 모델
intent = IntentModel(model_name='real_chat.h5', proprocess=p)

# 개체명 인식 모델
ner = NerModel(model_name='ner_model.h5', proprocess=p) # chat_ner로 수정 !!!


def to_client(conn, addr, params):
    db = params['db']

    try:
        db.connect()  # 디비 연결

        # 데이터 수신
        read = conn.recv(2048)  # 수신 데이터가 있을 때 까지 블로킹
        print('===========================')
        print('Connection from: %s' % str(addr))

        if read is None or not read:
            # 클라이언트 연결이 끊어지거나, 오류가 있는 경우
            print('클라이언트 연결 끊어짐')
            exit(0)


        # json 데이터로 변환
        recv_json_data = json.loads(read.decode())
        print("데이터 수신 : ", recv_json_data)
        query = recv_json_data['Query']

        # 의도 파악
        intent_predict = intent.predict_class(query)
        intent_name = intent.labels[intent_predict]

        # 개체명 파악
        ner_predicts = ner.predict(query)
        ner_tags = ner.predict_tags(query)


        # 답변 검색
        try:
            f = FindAnswer(db)
            answer_text, answer_image = f.search(intent_name, ner_tags)
            answer = f.tag_to_word(ner_predicts, answer_text)

        except:
            answer = "죄송해요 무슨 말인지 모르겠어요. 조금 더 공부 할게요."
            answer_image = None

        send_json_data_str = {
            "Query" : query,
            "Answer": answer,
            "AnswerImageUrl" : answer_image,
            "Intent": intent_name,
            "NER": str(ner_predicts)
        }
        message = json.dumps(send_json_data_str)
        conn.send(message.encode())

    except Exception as ex:
        print(ex)

    finally:
        if db is not None: # db 연결 끊기
            db.close()
        conn.close()


if __name__ == '__main__':

    # 질문/답변 학습 디비 연결 객체 생성
    db = Database(
        host='localhost', user='root', password='aiproject', db_name='chatbot'
    )
    print("DB 접속")

    port = 5050
    listen = 100

    # 봇 서버 동작
    bot = BotServer(port, listen)
    bot.create_sock()
    print("bot start")

    while True:
        conn, addr = bot.ready_for_client()
        params = {
            "db": db
        }

        client = threading.Thread(target=to_client, args=(
            conn,
            addr,
            params
        ))
        client.start()


DB 접속
bot start
